# 3.5 Streams

In [1]:
; define as macros to override Schemes eager evalutation
(define-syntax delay
               [(delay ?exp) (memo-proc (lambda() ?exp))])
(define-syntax cons-stream
               [(cons-stream ?a ?b) (cons ?a (delay ?b))])

(define the-empty-stream '())

(define (stream-null? s)
  (eq? s the-empty-stream))
(define (stream-car stream)
  (car stream))
(define (stream-cdr stream)
  (force (cdr stream)))

(define (stream-ref s n)
  (if (= n 0)
      (stream-car s)
      (stream-ref (stream-cdr s) (- n 1))))

(define (stream-map proc s)
  (if (stream-null? s)
      the-empty-stream
      (cons-stram
       (proc (stream-car s))
       (stream-map proc (stream-cdr s)))))

(define (stream-for-each proc s)
  (if (stream-null? s)
      'done
      (begin
       (proc (stream-car s))
       (stream-for-each proc
                        (stream-cdr s)))))

(define (stream-filter pred stream)
  (cond ((stream-null? stream)
         the-empty-stream)
        ((pred (stream-car stream))
         (cons-stream
          (stream-car stream)
          (stream-filter
           pred
           (stream-cdr stream))))
        (else (stream-filter
               pred
               (stream-cdr stream)))))

(define (stream-enumerate-interval low high)
  (if (> low high)
      the-empty-stream
      (cons-stream
       low
       (stream-enumerate-interval (+ low 1)
                                  high))))

(define (force delayed-object)
  (delayed-object))

(define (memo-proc proc)
  (let ((already-run? #f)
        (result #f))
    (lambda ()
      (if (not already-run?)
          (begin (set! result (proc))
                 (set! already-run? #t)
                 result)
          result))))

(define (display-stream s)
  (stream-for-each display-line s))

(define (display-line x)
  (newline)
  (display x))

## 3.50

In [2]:
; take proc of n args along with n streams
; apply proc to to all elements of each stream
(define (stream-map proc . argstreams)
  (if (stream-null? (car argstreams))
      the-empty-stream
      (cons-stream
       (apply proc (map stream-car argstreams))
       (apply stream-map
              (cons proc
                    (map stream-cdr
                         argstreams))))))

## 3.51

In [3]:
; prints and returns argument
(define (show x)
  (display-line x)
  x)

(define x
  (stream-map
   show
   (stream-enumerate-interval 0 10)))


0

In [4]:
(stream-ref x 5)


1
2
3
4
5

5

In [5]:
(stream-ref x 7)


6
7

7

## 3.52

In [6]:
(define sum 0)

sum

0

In [7]:
(define (accum x)
  (set! sum (+ x sum))
  sum)

sum

0

In [8]:
(define seq
  (stream-map
   accum
   (stream-enumerate-interval 1 20)))

sum

1

In [9]:
(define y (stream-filter even? seq))

sum

6

In [10]:
(define z
  (stream-filter
   (lambda (x)
     (= (remainder x 5) 0)) seq))

sum

10

In [11]:
(stream-ref y 7)

136

In [12]:
sum

136

In [13]:
(display-stream z)


10
15
45
55
105
120
190
210

done

In [14]:
sum

210

The results above would be different if `delay` did not use memoization through `memo-proc`. This is because `sum` is mutated each time `seq` is evaluated.`

# 3.5.2 Infinite Streams

In [6]:
(define ones (cons-stream 1 ones))

(define (add-streams s1 s2)
  (stream-map + s1 s2))

(define integers
  (cons-stream 1 (add-streams ones integers)))

(define fibs
  (cons-stream
   0 (cons-stream
      1 (add-streams
         (stream-cdr fibs) fibs))))

(define (scale-stream stream factor)
  (stream-map
   (lambda (x) (* x factor))
   stream))

(define primes
  (cons-stream
   2 (stream-filter
      prime? (integers-starting-from 3))))

; check whether n is divisible by a prime less than or equal to sqrt(n)
(define (prime? n)
  (define (iter ps)
    (cond ((> (square (stream-car ps)) n) #t)
          ((divisible? n (stream-car ps)) #f)
          (else (iter (stream-cdr ps)))))
  (iter primes))

## 3.53

```scheme
(define s (cons-stream 1 (add-streams s s)))
```
`s` is an infinite stream of powers of 2: `1, 2, 4, 8, 16, 32 ...`

It is equivalent to the `double` stream defined earlier in the chapter:
```scheme
(define double
  (cons-stream 1 (scale-stream double 2)))
```

## 3.54

In [7]:
(define (mul-streams s1 s2)
  (stream-map * s1 s2))

(define factorials
  (cons-stream 1 (mul-streams factorials integers)))

## 3.55

In [8]:
(define (partial-sums stream)
   (cons-stream (stream-car stream)
                (add-streams (stream-cdr stream)
                             (partial-sums stream))))

## 3.56

In [9]:
(define (merge s1 s2)
  (cond ((stream-null? s1) s2)
        ((stream-null? s2) s1)
        (else
         (let ((s1car (stream-car s1))
               (s2car (stream-car s2)))
           (cond ((< s1car s2car)
                  (cons-stream
                   s1car
                   (merge (stream-cdr s1)
                          s2)))
                 ((> s1car s2car)
                  (cons-stream
                   s2car
                   (merge s1
                          (stream-cdr s2))))
                 (else
                  (cons-stream
                   s1car
                   (merge
                    (stream-cdr s1)
                    (stream-cdr s2)))))))))

(define S (cons-stream
           1
           (merge (scale-stream S 2) (merge (scale-stream S 3)
                                            (scale-stream S 5)))))

## 3.57
```scheme
(define fibs
  (cons-stream
   0 (cons-stream
      1 (add-streams
         (stream-cdr fibs) fibs))))
```
The streams definition of `fibs` will perform $n-1$ additions when computing the $n^{th}$ Fibonacci number.

Each term $Fib _n$ is computed by $sum(Fib _{n-1}, Fib_{n-2})$. Due to memoization in `delay`, recalling the previous terms $Fib _{n-1}, Fib_{n-2}$ requires no additions. Therefore, each term *after the first* requires **one addition** (the first term is given in the definition of `fibs`).

Without memoization `delay`, computing $Fib _n$ will require re-calculating $Fib _{n-1}$ and $Fib_{n-2}$ instead of recalling the memoized values:

$Fib _1 = 1$

$T(Fib _{n-1}) = O(2^{n-1})$

$T(Fib _n) = T(Fib _n-1) + T(Fib _n-2) = O(2^{n-1}) + O(2^{n-2}) = O(2^n)$


## 3.58

In [10]:
(define (expand num den radix)
  (cons-stream
   (quotient (* num radix) den)
   (expand (remainder (* num radix) den)
           den
           radix)))

; print first n values of stream 
(define (display-stream-n s n)
  (define (iter i)
    (if (< i n)
        (begin 
         (display (stream-ref s i))
         (newline)
         (iter (+ i 1)))))
  (iter 0))

`expand` produces the digits of $\frac{num}{den}$ in base `radix`:

In [20]:
(display-stream-n (expand 1 7 10) 5)

1
4
2
8
5


#f

In [21]:
(float (/ 1 7))

0.14285714285714285

In [22]:
(display-stream-n (expand 3 8 10) 5)

3
7
5
0
0


#f

In [23]:
(float (/ 3 8))

0.375

## 3.59

In [24]:
(define (integrate-series s)
  (stream-map / s integers))

(define exp-series
  (cons-stream
   1 (integrate-series exp-series)))

(define cosine-series
  (cons-stream 1 (integrate-series sine-series)))

(define sine-series
  (cons-stream
   0 (scale-stream (integrate-series cosine-series) -1)))

## 3.60

In [25]:
(define (mul-series s1 s2)
  (cons-stream
   ; multiply first coeffs to create next element of series
   (* (stream-car s1) (stream-car s2))
   ; multiply remaining coeffs in s1 by first coeff in s2
   (add-streams (scale-stream (stream-cdr s1)
                              (stream-car s2))
                (mul-series s1 (stream-cdr s2)))))

(define cos^2+sin^2
  (add-streams (mul-series cosine-series cosine-series)
               (mul-series sine-series sine-series)))

(display-stream-n cos^2+sin^2 3)

1
0
0


#f

## 3.61

$S$ = power series whose constant term is $1$.

The power series $\frac{1}{S}$ is the series $X$ such that $SX=1$.

$S=1+S_R$ where $S_R$ is the part of $S$ *after* the constant term.

$SX=1$

$(1+S_R)X=1$

$X+S_RX=1$

$X=1-S_RX$


In [26]:
; computes 1/S for power series S w/constant term 1
(define (invert-unit-series s)
  (cons-stream 1 (scale-stream
                  (mul-series (stream-cdr s)
                              (invert-unit-series s))
                  -1)))

## 3.62

In [56]:
(define (div-series num-s denom-s)
  (let ((denom-constant (stream-car denom-s)))
    (if (zero? denom-constant)
        (error "div-series" "Denominator must have non-zero constant term")
        (mul-series
         num-s
         (scale-stream (invert-unit-series
                        (scale-stream denom-s (/ 1 denom-constant)))
                       denom-constant)))))

(define tan-series
  (div-series sine-series cosine-series))

(display-stream-n tan-series 5)

0
-1
0
-1/3
0


#f

## 3.63
```scheme
; with local `guesses` variable
(define (sqrt-stream x)
  (define guesses
    (cons-stream
     1.0 (stream-map
          (lambda (guess)
            (sqrt-improve guess x))
          guesses)))
  guesses)

; without local `guesses` variable
(define (sqrt-stream x)
  (cons-stream
   1.0
   (stream-map (lambda (guess)
                 (sqrt-improve guess x))
               (sqrt-stream x))))
```

Defining `guesses` means that the same stream is reused when generating further values. Due to the memoization in `delay`, previously computed values of the stream are not re-computed. Without `guesses` a new, yet to be memoized, stream is created each time another value of the stream is generated.

If `delay` didn't implement memoization, there would be no difference in efficiency between the two procedures as the `stream-map` call will force the evaluation of the `guesses` stream.

## 3.64

In [27]:
(define (sqrt-improve guess x)
  (define (average x y) (/ (+ x y) 2))
  (average guess (/ x guess)))

(define (sqrt-stream x)
  (define guesses
    (cons-stream
     1.0 (stream-map
          (lambda (guess)
            (sqrt-improve guess x))
          guesses)))
  guesses)

(define (stream-limit s tolerance)
  (cond ((stream-null? s) '())
        ((stream-null? (stream-cdr s)) (stream-car s))
        (else (let ((v1 (stream-ref s 0))
                    (v2 (stream-ref s 1)))
                (if (> tolerance (abs (- v1 v2)))
                    v2
                    (stream-limit (stream-cdr s) tolerance))))))

(define (sqrt-approx x tolerance)
  (stream-limit (sqrt-stream x) tolerance))

In [76]:
(sqrt-approx 2 0.1)

1.4166666666666665

In [77]:
(sqrt-approx 2 0.01)

1.4142156862745097

In [78]:
(sqrt-approx 2 0.001)

1.4142135623746899

## 3.65
$ln2 = 1 - \frac{1}{2}+\frac{1}{3}-\frac{1}{4}...$

In [84]:
(define (ln-summands n)
  (cons-stream
   (/ 1.0 n)
   (stream-map - (ln-summands (+ n 1)))))

(define ln2-stream
  (partial-sums (ln-summands 1)))

(display-stream-n ln2-stream 3)

1.0
0.5
0.8333333333333333


#f

## 3.66

In [11]:
; stream of element pairs between s and t
; (s0,t0) (s0,t1) (s0, t2) ...
;         (s1, t1) (s1, t2) ...
;                  (s2, t2) ...
;                           ...
(define (pairs s t)
  (cons-stream
   (list (stream-car s) (stream-car t))
   (interleave
    (stream-map (lambda (x)
                  (list (stream-car s) x))
                (stream-cdr t))
    (pairs (stream-cdr s) (stream-cdr t)))))

; combines two streams by taking elements alternately from each
(define (interleave s1 s2)
  (if (stream-null? s1)
      s2
      (cons-stream
       (stream-car s1)
       (interleave s2 (stream-cdr s1)))))

(define int-pairs (pairs integers integers))
(display-stream-n int-pairs 20)

(1 1)
(1 2)
(2 2)
(1 3)
(2 3)
(1 4)
(3 3)
(1 5)
(2 4)
(1 6)
(3 4)
(1 7)
(2 5)
(1 8)
(4 4)
(1 9)
(2 6)
(1 10)
(3 5)
(1 11)


#f

After the first pair `(1, 1)` a pair `(1, n)` occurs every **second** element. Therefore `(1, 100)` will have **197** preceding pairs.

In other words, `(1, 100)` occurs at index 197:

In [29]:
(stream-ref int-pairs 197)

(1 100)

`(2, 2)` pairs `(2, n)` appear every **4** elements -> $2^2$

`(3, 3)` pairs `(3, n)` appear every **8** elements -> $2^3$ 

Let $Z(m, n)$ be the index of pair $(m, n)$:

$Z(m, n)=2^m - 2$ for $m = n$ 

$Z(m, n)=2^m(n-m)+2^{(m-1)}-2$ for $m < n$ 

## 3.67

In [ ]:
(define (pairs s t)
  (cons-stream
   (list (stream-car s) (stream-car t))
   (interleave
    (stream-map
     (lambda (x)
       (list (stream-car s) x))
     (stream-cdr t))
    (interleave
     (stream-map
      (lambda (x)
        (list (stream-car t) x))
      (stream-cdr s))
     (pairs (stream-cdr s) (stream-cdr t))))))

## 3.68

Louis proposal will not work as the procedure will never terminate. The second argument to `interleave` is a recursive call to `pairs` - leading to infinite recursion as`interleave` requires the first value of both streams to be evaluated. 

# 3.69

In [ ]:
(define (square x) (* x x))

(define (triples s t u)
  (cons-stream
   (list (stream-car s) (stream-car t) (stream-car u))
   (interleave
    (stream-map (lambda (x)
                  (append (list (stream-car s)) x))
                (stream-cdr (pairs t u)))
    (triples (stream-cdr s)
             (stream-cdr t)
             (stream-cdr u)))))

(define (is-pythag-triple triple)
  (= (+ (square (car triple))
        (square (cadr triple)))
     (square (caddr triple))))

(define pythagorean-triples
  (stream-filter is-pythag-triple
                 (triples integers integers integers)))

(display-stream-n pythagorean-triples 4)

(3 4 5)
(6 8 10)
(5 12 13)
(9 12 15)
